In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import os
from datetime import datetime
import pytz
from pathlib import Path
import sklearn as sk
import tensorflow as tf
import torch
import torch.nn as nn
from torchvision.io import read_image
import time
import timeit
import random
import torchvision.models as models
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
import sklearn.metrics
import csv
import timeit

%matplotlib inline

In [ ]:
# mount google drive to use as data repo
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls 'drive/MyDrive/DS5220: Final Project/logs/resnet18'

20240425_170226_logs/  20240425_170447_logs/  20240425_191945_logs/  20240425_202833_logs/
20240425_170240_logs/  20240425_174634_logs/  20240425_195704_logs/  20240425_210059_logs/
20240425_170436_logs/  20240425_180207_logs/  20240425_201810_logs/


In [ ]:
BASE_LOGS_PATH = 'drive/MyDrive/DS5220: Final Project/logs/resnet18'

In [ ]:
# https://pytorch.org/docs/stable/notes/mps.html

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# enable memory history, which will
# add tracebacks and event history to snapshots
#torch.cuda.memory._record_memory_history()

# add to end of training
#torch.cuda.memory._dump_snapshot("my_snapshot.pickle")


Using cuda device


In [ ]:
###################################################
CIFAR100_ROOT_PATH='drive/MyDrive/DS5220: Final Project'  # Modify this line with the path to the folder where folder "cifar-10-batches-py" locate
###################################################

class Cifar100():
    def __init__(self,
                 calculate_mean_and_std = False):



        if calculate_mean_and_std:
            self.mean, self.std = self.calculate_mean_and_std()
        else:
            self.mean, self.std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

        self.BATCH_SIZE = 128
        self.transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)])

        self.train_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                download=True,
                                train=True,
                                transform=self.transform)

        self.eval_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                train=False,
                                transform=self.transform)

        self.train_data_loader = DataLoader(dataset=self.train_dataset,
                               num_workers=0,
                               batch_size=self.BATCH_SIZE,
                               shuffle=True)

        self.eval_data_loader = DataLoader(dataset=self.eval_dataset,
                              num_workers=0,
                              batch_size=self.BATCH_SIZE,
                              shuffle=False)

    def calculate_mean_and_std(self):

        train_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                download=True,
                                train=True)
        # stick all the images together to form a 1600000 X 32 X 3 array
        x = np.concatenate([np.asarray(train_dataset[i][0]) for i in range(len(train_dataset))])
        # calculate the mean and std along the (0, 1) axes
        _mean = np.mean(x, axis=(0, 1))/255
        _std = np.std(x, axis=(0, 1))/255
        _mean = _mean.tolist()
        _std = _std.tolist()

        return _mean, _std




class Cifar100WithAugmentation():
    def __init__(self, calculate_mean_and_std=False):
        CIFAR100_ROOT_PATH = 'drive/MyDrive/DS5220: Final Project'  # Modify this path accordingly

        if calculate_mean_and_std:
            self.mean, self.std = self.calculate_mean_and_std()
        else:
            self.mean, self.std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

        self.BATCH_SIZE = 128
        # data augmentation and normalization for training
        self.train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),  # pad 4 pixels on each side and randomly crop a 32x32 image
            transforms.RandomHorizontalFlip(),     # randomly flip the image horizontally
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

        # normalization for evaluation (without augmentation)
        self.eval_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

        self.train_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                               download=True,
                                               train=True,
                                               transform=self.train_transform)

        self.eval_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                              train=False,
                                              transform=self.eval_transform)

        self.train_data_loader = DataLoader(dataset=self.train_dataset,
                                            num_workers=4,
                                            batch_size=self.BATCH_SIZE,
                                            shuffle=True)

        self.eval_data_loader = DataLoader(dataset=self.eval_dataset,
                                           num_workers=4,
                                           batch_size=self.BATCH_SIZE,
                                           shuffle=False)

    def calculate_mean_and_std(self):
        train_dataset = CIFAR100(root='drive/MyDrive/DS5220: Final Project',
                                          download=True,
                                          train=True)
        # concatenate all the images to compute the mean and std
        x = np.concatenate([np.asarray(train_dataset[i][0]) for i in range(len(train_dataset))])
        # compute mean and std along the (0, 1) axes
        mean = np.mean(x, axis=(0, 1)) / 255
        std = np.std(x, axis=(0, 1)) / 255
        return mean.tolist(), std.tolist()


cf100 = Cifar100(calculate_mean_and_std=True)
cf100_w_aug = Cifar100WithAugmentation(calculate_mean_and_std=True)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
config = {
    'dataloader' : cf100_w_aug,
    'model': models.resnet18(),
    'epochs': 70,
    'optimizer': 'SGD',
    'learning_rate': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'loss': nn.CrossEntropyLoss(),
    'log_dir': os.path.join(BASE_LOGS_PATH, time.strftime('%Y%m%d_%H%M%S') + '_logs'),
    'device' : device,
    'run_time' : None
}

model = config['model']
model.to(device)
EPOCHS = config['epochs']
criterion = config['loss']
optimizer = torch.optim.SGD(model.parameters(),
                            lr=config['learning_rate'],
                            momentum=config['momentum'],
                            weight_decay=config['weight_decay'])

# create log directory
if not os.path.exists(config['log_dir']):
    os.makedirs(config['log_dir'])
    print(f"Directory '{config['log_dir']}' created")
else:
    print(f"Directory '{config['log_dir']}' already exists")



metrics_path = os.path.join(config['log_dir'], 'training_metrics.csv')
with open(metrics_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # write the header row
    writer.writerow(['Epoch', 'Train Loss', 'Train Accuracy', 'Eval Loss', 'Eval Accuracy', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Epoch Time (seconds)'])
    training_start_time = timeit.default_timer()
    for epoch in range(config['epochs']):
        start_time = timeit.default_timer()
        train_losses = []
        train_accuracies = []

        model.train()
        for images, labels in config['dataloader'].train_data_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == labels).float().mean().item()
            train_accuracies.append(accuracy)

        eval_losses = []
        eval_accuracies = []
        top1_accuracies = []
        top5_accuracies = []
        model.eval()
        with torch.no_grad():
            for images, labels in config['dataloader'].eval_data_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                eval_losses.append(loss.item())

                # calculate top-1 accuracy
                _, predicted = torch.max(outputs, 1)
                top1_accuracy = (predicted == labels).float().mean().item()
                top1_accuracies.append(top1_accuracy)

                # calculate top-5 accuracy
                _, top5_preds = outputs.topk(5, dim=1)
                top5_correct = top5_preds.eq(labels.view(-1, 1).expand_as(top5_preds))
                top5_accuracy = top5_correct.float().sum(1).ge(1).float().mean().item()
                top5_accuracies.append(top5_accuracy)

                eval_accuracies.append(top1_accuracy)
        # write model training info to csv
        end_time = timeit.default_timer()  # end timing the epoch
        epoch_duration = end_time - start_time
        writer.writerow([
            epoch + 1,
            np.mean(train_losses),
            np.mean(train_accuracies) * 100,
            np.mean(eval_losses),
            np.mean(eval_accuracies) * 100,
            np.mean(top1_accuracies) * 100,
            np.mean(top5_accuracies) * 100,
            epoch_duration
        ])
        print(f"Epoch {epoch+1}/{config['epochs']}")
        print(f"Train Loss: {np.mean(train_losses):.4f}, Train Accuracy: {np.mean(train_accuracies) * 100:.2f}%")
        print(f"Eval Loss: {np.mean(eval_losses):.4f}, Eval Accuracy: {np.mean(eval_accuracies) * 100:.2f}%, Top-1 Accuracy: {np.mean(top1_accuracies) * 100:.2f}%, Top-5 Accuracy: {np.mean(top5_accuracies) * 100:.2f}%")
    training_end_time = timeit.default_timer()  # end timing the epoch
    training_duration = training_end_time - training_start_time
    config['run_time'] = training_duration

config_path = os.path.join(config['log_dir'], 'config.txt')
# output config file to directory
with open(config_path, 'w') as f:
    for key, value in config.items():
        f.write(f'{key}: {value}\n')



torch.save(model.state_dict(), os.path.join(config['log_dir'], 'model_state_dict.pth'))
torch.save(model, os.path.join(config['log_dir'], 'complete_model.pth'))


Directory 'drive/MyDrive/DS5220: Final Project/logs/resnet18/20240425_202833_logs' created
Epoch 1/70
Train Loss: 3.9702, Train Accuracy: 11.02%
Eval Loss: 3.3869, Eval Accuracy: 18.84%, Top-1 Accuracy: 18.84%, Top-5 Accuracy: 45.02%
Epoch 2/70
Train Loss: 3.2743, Train Accuracy: 20.34%
Eval Loss: 3.0304, Eval Accuracy: 25.51%, Top-1 Accuracy: 25.51%, Top-5 Accuracy: 55.17%
Epoch 3/70
Train Loss: 2.9752, Train Accuracy: 26.11%
Eval Loss: 2.8057, Eval Accuracy: 29.82%, Top-1 Accuracy: 29.82%, Top-5 Accuracy: 59.82%
Epoch 4/70
Train Loss: 2.7501, Train Accuracy: 29.82%
Eval Loss: 2.6492, Eval Accuracy: 32.51%, Top-1 Accuracy: 32.51%, Top-5 Accuracy: 63.93%
Epoch 5/70
Train Loss: 2.5905, Train Accuracy: 33.18%
Eval Loss: 2.5279, Eval Accuracy: 35.08%, Top-1 Accuracy: 35.08%, Top-5 Accuracy: 66.79%
Epoch 6/70
Train Loss: 2.4349, Train Accuracy: 36.13%
Eval Loss: 2.6001, Eval Accuracy: 34.55%, Top-1 Accuracy: 34.55%, Top-5 Accuracy: 65.27%
Epoch 7/70
Train Loss: 2.3314, Train Accuracy: 38.5

In [ ]:
class ResNet18WithDropout(models.ResNet):
    def __init__(self, dropout_rate=0.5, num_classes=100):
        super(ResNet18WithDropout, self).__init__(block=models.resnet.BasicBlock, layers=[2, 2, 2, 2], num_classes=num_classes)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc = nn.Sequential(
            nn.Linear(self.fc.in_features, 512),
            nn.ReLU(),
            self.dropout,
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.fc(x)

        return x


dropout_rate = 0.5
num_classes = 100

config = {
    'dataloader' : cf100_w_aug,
    'model': ResNet18WithDropout(dropout_rate=dropout_rate, num_classes=num_classes),
    'epochs': 40,
    'optimizer': 'SGD',
    'learning_rate': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'loss': nn.CrossEntropyLoss(),
    'log_dir': os.path.join(BASE_LOGS_PATH, time.strftime('%Y%m%d_%H%M%S') + '_logs'),
    'device' : device,
    'run_time' : None
}

model = config['model']
model.to(device)
EPOCHS = config['epochs']
criterion = config['loss']
optimizer = torch.optim.SGD(model.parameters(),
                            lr=config['learning_rate'],
                            momentum=config['momentum'],
                            weight_decay=config['weight_decay'])

# create log directory
if not os.path.exists(config['log_dir']):
    os.makedirs(config['log_dir'])
    print(f"Directory '{config['log_dir']}' created")
else:
    print(f"Directory '{config['log_dir']}' already exists")



metrics_path = os.path.join(config['log_dir'], 'training_metrics.csv')
with open(metrics_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # write the header row
    writer.writerow(['Epoch', 'Train Loss', 'Train Accuracy', 'Eval Loss', 'Eval Accuracy', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Epoch Time (seconds)'])
    training_start_time = timeit.default_timer()
    for epoch in range(config['epochs']):
        start_time = timeit.default_timer()
        train_losses = []
        train_accuracies = []

        model.train()
        for images, labels in config['dataloader'].train_data_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == labels).float().mean().item()
            train_accuracies.append(accuracy)

        eval_losses = []
        eval_accuracies = []
        top1_accuracies = []
        top5_accuracies = []
        model.eval()
        with torch.no_grad():
            for images, labels in config['dataloader'].eval_data_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                eval_losses.append(loss.item())

                # calculate top-1 accuracy
                _, predicted = torch.max(outputs, 1)
                top1_accuracy = (predicted == labels).float().mean().item()
                top1_accuracies.append(top1_accuracy)

                # calculate top-5 accuracy
                _, top5_preds = outputs.topk(5, dim=1)
                top5_correct = top5_preds.eq(labels.view(-1, 1).expand_as(top5_preds))
                top5_accuracy = top5_correct.float().sum(1).ge(1).float().mean().item()
                top5_accuracies.append(top5_accuracy)

                eval_accuracies.append(top1_accuracy)
        # write model training info to csv
        end_time = timeit.default_timer()  # end timing the epoch
        epoch_duration = end_time - start_time
        writer.writerow([
            epoch + 1,
            np.mean(train_losses),
            np.mean(train_accuracies) * 100,
            np.mean(eval_losses),
            np.mean(eval_accuracies) * 100,
            np.mean(top1_accuracies) * 100,
            np.mean(top5_accuracies) * 100,
            epoch_duration
        ])
        print(f"Epoch {epoch+1}/{config['epochs']}")
        print(f"Train Loss: {np.mean(train_losses):.4f}, Train Accuracy: {np.mean(train_accuracies) * 100:.2f}%")
        print(f"Eval Loss: {np.mean(eval_losses):.4f}, Eval Accuracy: {np.mean(eval_accuracies) * 100:.2f}%, Top-1 Accuracy: {np.mean(top1_accuracies) * 100:.2f}%, Top-5 Accuracy: {np.mean(top5_accuracies) * 100:.2f}%")
    training_end_time = timeit.default_timer()  # end timing the epoch
    training_duration = training_end_time - training_start_time
    config['run_time'] = training_duration

config_path = os.path.join(config['log_dir'], 'config.txt')
# output config file to directory
with open(config_path, 'w') as f:
    for key, value in config.items():
        f.write(f'{key}: {value}\n')



torch.save(model.state_dict(), os.path.join(config['log_dir'], 'model_state_dict.pth'))
torch.save(model, os.path.join(config['log_dir'], 'complete_model.pth'))



Directory 'drive/MyDrive/DS5220: Final Project/logs/resnet18/20240425_210059_logs' created
Epoch 1/40
Train Loss: 4.0807, Train Accuracy: 7.36%
Eval Loss: 3.6172, Eval Accuracy: 13.92%, Top-1 Accuracy: 13.92%, Top-5 Accuracy: 38.29%
Epoch 2/40
Train Loss: 3.5773, Train Accuracy: 14.75%
Eval Loss: 3.2885, Eval Accuracy: 20.08%, Top-1 Accuracy: 20.08%, Top-5 Accuracy: 47.61%
Epoch 3/40
Train Loss: 3.3084, Train Accuracy: 19.67%
Eval Loss: 3.0545, Eval Accuracy: 23.51%, Top-1 Accuracy: 23.51%, Top-5 Accuracy: 54.20%
Epoch 4/40
Train Loss: 3.1140, Train Accuracy: 23.10%
Eval Loss: 2.9297, Eval Accuracy: 27.34%, Top-1 Accuracy: 27.34%, Top-5 Accuracy: 57.93%
Epoch 5/40
Train Loss: 2.9557, Train Accuracy: 26.16%
Eval Loss: 2.7879, Eval Accuracy: 29.00%, Top-1 Accuracy: 29.00%, Top-5 Accuracy: 61.47%
Epoch 6/40
Train Loss: 2.8289, Train Accuracy: 28.52%
Eval Loss: 2.7111, Eval Accuracy: 31.06%, Top-1 Accuracy: 31.06%, Top-5 Accuracy: 62.72%
Epoch 7/40
Train Loss: 2.7229, Train Accuracy: 30.45

In [ ]:
class ResNet18WithDropout(models.ResNet):
    def __init__(self, dropout_rate=0.4, num_classes=100):

        super(ResNet18WithDropout, self).__init__(block=models.resnet.BasicBlock, layers=[2, 2, 2, 2], num_classes=num_classes)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc = nn.Sequential(
            nn.Linear(self.fc.in_features, 512),
            nn.ReLU(),
            self.dropout,
            nn.Linear(512, num_classes)
        )

    def forward(self, x):

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.fc(x)

        return x


dropout_rate = 0.4
num_classes = 100

config = {
    'dataloader' : cf100_w_aug,
    'model': ResNet18WithDropout(dropout_rate=dropout_rate, num_classes=num_classes),
    'epochs': 70,
    'optimizer': 'SGD',
    'learning_rate': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'loss': nn.CrossEntropyLoss(),
    'log_dir': os.path.join(BASE_LOGS_PATH, time.strftime('%Y%m%d_%H%M%S') + '_logs'),
    'device' : device,
    'run_time' : None
}

model = config['model']
model.to(device)
EPOCHS = config['epochs']
criterion = config['loss']
optimizer = torch.optim.SGD(model.parameters(),
                            lr=config['learning_rate'],
                            momentum=config['momentum'],
                            weight_decay=config['weight_decay'])

# create log directory
if not os.path.exists(config['log_dir']):
    os.makedirs(config['log_dir'])
    print(f"Directory '{config['log_dir']}' created")
else:
    print(f"Directory '{config['log_dir']}' already exists")



metrics_path = os.path.join(config['log_dir'], 'training_metrics.csv')
with open(metrics_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # write the header row
    writer.writerow(['Epoch', 'Train Loss', 'Train Accuracy', 'Eval Loss', 'Eval Accuracy', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Epoch Time (seconds)'])
    training_start_time = timeit.default_timer()
    for epoch in range(config['epochs']):
        start_time = timeit.default_timer()
        train_losses = []
        train_accuracies = []

        model.train()
        for images, labels in config['dataloader'].train_data_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == labels).float().mean().item()
            train_accuracies.append(accuracy)

        eval_losses = []
        eval_accuracies = []
        top1_accuracies = []
        top5_accuracies = []
        model.eval()
        with torch.no_grad():
            for images, labels in config['dataloader'].eval_data_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                eval_losses.append(loss.item())

                # calculate top-1 accuracy
                _, predicted = torch.max(outputs, 1)
                top1_accuracy = (predicted == labels).float().mean().item()
                top1_accuracies.append(top1_accuracy)

                # calculate top-5 accuracy
                _, top5_preds = outputs.topk(5, dim=1)
                top5_correct = top5_preds.eq(labels.view(-1, 1).expand_as(top5_preds))
                top5_accuracy = top5_correct.float().sum(1).ge(1).float().mean().item()
                top5_accuracies.append(top5_accuracy)

                eval_accuracies.append(top1_accuracy)
        # write model training info to csv
        end_time = timeit.default_timer()  # end timing the epoch
        epoch_duration = end_time - start_time
        writer.writerow([
            epoch + 1,
            np.mean(train_losses),
            np.mean(train_accuracies) * 100,
            np.mean(eval_losses),
            np.mean(eval_accuracies) * 100,
            np.mean(top1_accuracies) * 100,
            np.mean(top5_accuracies) * 100,
            epoch_duration
        ])
        print(f"Epoch {epoch+1}/{config['epochs']}")
        print(f"Train Loss: {np.mean(train_losses):.4f}, Train Accuracy: {np.mean(train_accuracies) * 100:.2f}%")
        print(f"Eval Loss: {np.mean(eval_losses):.4f}, Eval Accuracy: {np.mean(eval_accuracies) * 100:.2f}%, Top-1 Accuracy: {np.mean(top1_accuracies) * 100:.2f}%, Top-5 Accuracy: {np.mean(top5_accuracies) * 100:.2f}%")
    training_end_time = timeit.default_timer()  # end timing the epoch
    training_duration = training_end_time - training_start_time
    config['run_time'] = training_duration

config_path = os.path.join(config['log_dir'], 'config.txt')
# output config file to directory
with open(config_path, 'w') as f:
    for key, value in config.items():
        f.write(f'{key}: {value}\n')



torch.save(model.state_dict(), os.path.join(config['log_dir'], 'model_state_dict.pth'))
torch.save(model, os.path.join(config['log_dir'], 'complete_model.pth'))



Directory 'drive/MyDrive/DS5220: Final Project/logs/resnet18/20240426_015532_logs' created


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1/70
Train Loss: 4.0576, Train Accuracy: 7.70%
Eval Loss: 3.5892, Eval Accuracy: 14.73%, Top-1 Accuracy: 14.73%, Top-5 Accuracy: 39.39%
Epoch 2/70
Train Loss: 3.5296, Train Accuracy: 15.56%
Eval Loss: 3.2594, Eval Accuracy: 20.55%, Top-1 Accuracy: 20.55%, Top-5 Accuracy: 48.39%
Epoch 3/70
Train Loss: 3.2646, Train Accuracy: 20.44%
Eval Loss: 3.0889, Eval Accuracy: 23.55%, Top-1 Accuracy: 23.55%, Top-5 Accuracy: 53.81%
Epoch 4/70
Train Loss: 3.0608, Train Accuracy: 23.88%
Eval Loss: 2.8151, Eval Accuracy: 29.08%, Top-1 Accuracy: 29.08%, Top-5 Accuracy: 59.88%
Epoch 5/70
Train Loss: 2.8754, Train Accuracy: 27.74%
Eval Loss: 2.6488, Eval Accuracy: 31.70%, Top-1 Accuracy: 31.70%, Top-5 Accuracy: 64.13%
Epoch 6/70
Train Loss: 2.7388, Train Accuracy: 30.16%
Eval Loss: 2.5780, Eval Accuracy: 33.29%, Top-1 Accuracy: 33.29%, Top-5 Accuracy: 65.04%
Epoch 7/70
Train Loss: 2.6217, Train Accuracy: 32.65%
Eval Loss: 2.5391, Eval Accuracy: 34.36%, Top-1 Accuracy: 34.36%, Top-5 Accuracy: 65.77%
